# Churn &mdash; Baseline Model

## Setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display, Markdown
plt.style.use("seaborn-darkgrid")
pd.set_option('display.max_columns', None)  

import sys, os, yaml

DATASET = "Churn"

COLAB = 'google.colab' in sys.modules
if COLAB:
    ROOT = f"/content/gdrive/MyDrive/datasets/{DATASET.replace(' ','_')}/"
else:
    ROOT = "./"

DEBUG = False
SEED = 666

C:\Users\Kuba\AppData\Local\Temp\ipykernel_2116\250077564.py:7: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-darkgrid")


In [3]:
if COLAB:
    from google.colab import drive
    if not os.path.isdir("/content/gdrive"):
        drive.mount("/content/gdrive")

## Load Dataset

In [4]:
df = pd.read_pickle(f"{ROOT}/data/data.pkl")
print(df.shape)
df.head(1)

(7032, 20)


,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,No,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No


In [5]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, train_size=0.60, random_state=SEED, stratify=df.Churn)
print(df_train.shape, df_test.shape)

(4219, 20) (2813, 20)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   Gender            7032 non-null   category
 1   SeniorCitizen     7032 non-null   category
 2   Partner           7032 non-null   category
 3   Dependents        7032 non-null   category
 4   Tenure            7032 non-null   int64   
 5   PhoneService      7032 non-null   category
 6   MultipleLines     7032 non-null   category
 7   InternetService   7032 non-null   category
 8   OnlineSecurity    7032 non-null   category
 9   OnlineBackup      7032 non-null   category
 10  DeviceProtection  7032 non-null   category
 11  TechSupport       7032 non-null   category
 12  StreamingTV       7032 non-null   category
 13  StreamingMovies   7032 non-null   category
 14  Contract          7032 non-null   category
 15  PaperlessBilling  7032 non-null   category
 16  PaymentMethod     7032 n

## Preprocessing Dataset

### Identify target and features ###

In [7]:
target = "Churn"
print(f"target : {target}")

cat_features = [c for c in df.select_dtypes("category").columns if c != target]
print(f"Categorical Features ({len(cat_features)}) : {cat_features}")

num_features = [c for c in df.select_dtypes(["int", "float"]).columns if c != target]
print(f"Numerical Features ({len(num_features)}) : {num_features}")

target : Churn
Categorical Features (16) : ['Gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
Numerical Features (3) : ['Tenure', 'MonthlyCharges', 'TotalCharges']


### Train/test split ###

In [8]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.40, stratify=df[target], random_state=SEED)
print(df_train.shape, df_test.shape)

(4219, 20) (2813, 20)


### Encode target ###

In [9]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
labelEncoder.fit(df_train[target])
y_train = labelEncoder.transform(df_train[target])
y_test = labelEncoder.transform(df_test[target])

In [10]:
df_train[target].head()

193      No
6108     No
3580    Yes
2195     No
3809     No
Name: Churn, dtype: category
Categories (2, object): ['No', 'Yes']

In [11]:
y_train[:5]

array([0, 0, 1, 0, 0])

In [12]:
labelEncoder.inverse_transform(y_train[:5])

array(['No', 'No', 'Yes', 'No', 'No'], dtype=object)

### Encode categorical features ###

In [13]:
from sklearn.preprocessing import OneHotEncoder

oneHotEncoder = OneHotEncoder()
oneHotEncoder.fit(df_train[cat_features])
x_cat_train = oneHotEncoder.transform(df_train[cat_features])
x_cat_test = oneHotEncoder.transform(df_test[cat_features])

In [14]:
df_cat_train = pd.DataFrame(x_cat_train.toarray(), columns=oneHotEncoder.get_feature_names_out())
df_cat_test = pd.DataFrame(x_cat_test.toarray(), columns=oneHotEncoder.get_feature_names_out())
print(df_cat_train.shape, df_cat_test.shape)
df_cat_train.head(1)

(4219, 36) (2813, 36)


,Gender_Female,Gender_Male,SeniorCitizen_No,SeniorCitizen_Yes,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_Yes,TechSupport_No,TechSupport_Yes,StreamingTV_No,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


### Scale / transform numerical features

In [15]:
from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()

standardScaler.fit(df_train[num_features])
x_num_train = standardScaler.transform(df_train[num_features])
x_num_test = standardScaler.transform(df_test[num_features])

In [16]:
df_num_train = pd.DataFrame(x_num_train, columns=standardScaler.get_feature_names_out())
df_num_test = pd.DataFrame(x_num_test, columns=standardScaler.get_feature_names_out())
print(df_num_train.shape, df_num_test.shape)
df_num_train.head(1)

(4219, 3) (2813, 3)


,Tenure,MonthlyCharges,TotalCharges
0,1.597275,1.487874,2.557942


### Construct dataframe for model features ###

In [18]:
df_model_train = pd.concat([df_cat_train, df_num_train], axis=1)
df_model_test = pd.concat([df_cat_test, df_num_test], axis=1)
print(df_model_train.shape, df_model_test.shape)

(4219, 39) (2813, 39)


## Model selection ##

In [29]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

classifiers = {
    "KNN" : KNeighborsClassifier(),
    "KNN (3)" : KNeighborsClassifier(3),
    "DT" : DecisionTreeClassifier(),
    "DT (max_depth=5" : DecisionTreeClassifier(max_depth=5),
    "LR" : LogisticRegression(max_iter=1000),
    "RF" : RandomForestClassifier(),
    "AdaBoost" : AdaBoostClassifier()
}

In [30]:
from sklearn.metrics import accuracy_score
for name, model in classifiers.items():
    model.fit(df_model_train, y_train)

    # Scoring on SEEN data - effectively "useless"
    y_pred = model.predict(df_model_train)
    train_accuracy = accuracy_score(y_train, y_pred)

    # Scoring on UNSEEN data - important
    y_pred = model.predict(df_model_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f"{name:20s} accuracy: train = {train_accuracy:.2%} test = {test_accuracy:.2%}")

KNN                  accuracy: train = 84.14% test = 75.72%
KNN (3)              accuracy: train = 87.08% test = 74.76%
DT                   accuracy: train = 99.79% test = 72.20%
DT (max_depth=5      accuracy: train = 80.28% test = 78.74%
LR                   accuracy: train = 80.47% test = 80.38%
RF                   accuracy: train = 99.79% test = 78.56%
AdaBoost             accuracy: train = 80.80% test = 79.88%


In [31]:
from sklearn.model_selection import cross_val_score
for name, model in classifiers.items():

    scores = cross_val_score(model, df_model_train, y_train, cv=10)

    print(f"{name:20s} accuracy CV = {scores.mean():.2%} std = {scores.std():.2%}")

KNN                  accuracy CV = 76.61% std = 1.64%
KNN (3)              accuracy CV = 74.99% std = 1.96%
DT                   accuracy CV = 72.20% std = 1.68%
DT (max_depth=5      accuracy CV = 78.38% std = 1.28%
LR                   accuracy CV = 80.21% std = 1.58%
RF                   accuracy CV = 78.57% std = 1.21%
AdaBoost             accuracy CV = 79.85% std = 1.60%
